# Tarea 1 - Árboles de decisión

### Grupo 36:
     - N. Farías
     - J. M. Varela

## 1. Objetivo

El objetivo de esta tarea es construir un clasificador de éxito/fracaso de estudiantes utilizando el algoritmo de árbol de decisión ID3 con variantes.

El éxito del aprendizaje se mide a través del accuracy. Es decir, la proporción de instancias correctamente clasificadas.

## 2. Diseño

## 2.1 Preprocesamiento de datos

* En la columna "Target", los valores "Enrolled" y "Graduate", fueron combinados en uno solo: "Success".  
* Todas las columnas con valores continuos (tipo de datos float64) fueron divididas en 5 intervalos de igual tamaño.  
* Si bien algunas columnas con valores enteros podrían dividirse en intervalos (por ejemplo "Age at enrollment"), se decidió no hacerlo porque la cantidad de valores únicos no es significativa.  
* Se dividió el conjunto de datos según las siguientes proporciones: 60% para entrenamiento, 20% para validación y 20% para evaluación.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from tarea_1 import ID3TreeClassifier

In [ ]:
# Carga de datos
data = pd.read_csv('data.csv', delimiter=';')

print(data.shape)
data.head()

(4424, 37)


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [ ]:
# Combinar 'enrolled' y 'graduate' en una sola categoría 'success'
data['Target'] = data['Target'].replace(['Enrolled', 'Graduate'], 'Success')

data['Target'].value_counts()

Success    3003
Dropout    1421
Name: Target, dtype: int64

In [ ]:
# Exploración inicial
print("Tipos de datos")
print(data.dtypes)
print("")
print("Cantidad de valores únicos")
print(data.nunique())

Tipos de datos
Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64

In [ ]:
# Columnas continuas
columnas_continuas = ['Previous qualification (grade)', 'Admission grade', 'Curricular units 1st sem (grade)',
                      'Curricular units 2nd sem (grade)', 'Unemployment rate', 'Inflation rate', 'GDP']

kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')

for col in columnas_continuas:
    data[col] = kbd.fit_transform(data[[col]]).astype(int)

data.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,1,1,19,12,...,0,0,0,0,0,0,1,2,3,Dropout
1,1,15,1,9254,1,1,3,1,1,3,...,0,6,6,6,3,0,3,0,3,Success
2,1,1,5,9070,1,1,1,1,37,37,...,0,6,0,0,0,0,1,2,3,Dropout
3,1,17,2,9773,1,1,1,1,38,37,...,0,6,10,5,3,0,1,0,0,Success
4,2,39,1,8014,0,1,0,1,37,38,...,0,6,6,6,3,0,3,0,3,Success


In [ ]:
# Division del dataset
train_data_pre, test_data = train_test_split(data, test_size=0.2, random_state=123)

train_data, validation_data = train_test_split(train_data_pre, test_size=0.25, random_state=123)

train_data.shape, validation_data.shape, test_data.shape

((2654, 37), (885, 37), (885, 37))

In [ ]:
X_train = train_data.drop(columns='Target')
y_train = train_data['Target']
X_validation = validation_data.drop(columns='Target')
y_validation = validation_data['Target']
X_test = test_data.drop(columns='Target')
y_test = test_data['Target']

## 2.2 Algoritmo

Se implementó el algoritmo de aprendizaje ID3 incorporando dos hiperparámetros que permiten evitar el sobreajuste:


*   **min_samples_split**: cantidad mínima necesaria para crear un nuevo nodo. Al momento de filtrar los ejemplos por el valor de un atributo, si la cantidad resultante es menor al parámetro, se trata de igual forma que si no hubiera ningún ejemplo. Es decir, se crea una hoja con la etiqueta mayoritaria dentro de los ejemplos del nodo actual.
*   **min_split_gain**: ganancia mínima requerida para partir por un atributo. Si la ganancia de ningún atributo alcanza ese valor, se trata de igual forma que si no quedaran atributos. Es decir, se crear una hoja con la etiqueta mayoritaria dentro de los ejemplos del nodo actual.

Para la selección de atributos se utilizan las funciones de ganancia y entropía vistas en el curso. En cada nodo de tipo atributo, se almacena además el valor correspondiente a la rama que determina el subconjunto de datos más grande. Esto permite que al momento de clasificar, si se encuentra un valor desconocido para un atributo, se pueda continuar por la rama del subconjunto más grande.



## 2.3 Evaluación

* Para la evaluación de la solución, se utilizó la métrica de accuracy, que es la proporción de predicciones correctas sobre el total de predicciones realizadas.   
* Se dividió el conjunto de datos en un conjunto de entrenamiento (60% de los datos), un conjunto de validación (20% de los datos) y un conjunto de prueba (20% de los datos).
* El conjunto de entrenamiento se utilizó para entrenar el árbol con distintas combinaciones de hiperparámetros, que se iban validando contra el conjunto de validación.
* Finalmente se entrenó un nuevo arbol con el conjunto de entrenamiento + validación, con los mejores hiperparámetros hallados previamente (los que daban mayor accuracy), y se evaluó en el conjunto de prueba.
* Se entrenaron los modelos DecisionTreeClassifier() y RandomForestClassifier() de scikit-learn con el conjunto de entrenamiento + validación, y se midió el accuracy sobre el conjunto de evaluación con el fin de tener una referencia para el rendimiento del modelo implementado.  

### Prueba de entrenamiento y evaluación inicial
Se realiza una prueba inicial con valores de hiperparámetros selecionados arbitrariamente

In [ ]:
decision_tree = ID3TreeClassifier(min_samples_split=10, min_split_gain=0.01)
decision_tree.fit(X_train, y_train)
validation_data_predictions = decision_tree.predict(X_validation)

# Calcular la precisión de las predicciones
accuracy = (validation_data_predictions == y_validation).mean()
accuracy

0.831638418079096

### Opcionalmente se puede imprimir el árbol resultante del entrenamiento

In [ ]:
decision_tree.print()

### Entranamiento con distintas combinaciones de hiperparámetros

In [ ]:
# Definir diferentes valores para los hiperparámetros
min_samples_split_values = [5, 10, 20, 40, 80]
min_split_gain_values = [0.01, 0.05, 0.1, 0.2, 0.3]

# Diccionario para almacenar los resultados
results = {}

# Evaluar el árbol de decisión para cada combinación de hiperparámetros
for min_samples in min_samples_split_values:
    for min_gain in min_split_gain_values:
        # Construir el árbol de decisión
        decision_tree = ID3TreeClassifier(min_samples_split=min_samples, min_split_gain=min_gain)
        decision_tree.fit(X_train, y_train)

        # Realizar predicciones en el conjunto de validación
        validation_data_predictions = decision_tree.predict(X_validation)

        # Calcular la precisión
        accuracy = (validation_data_predictions == y_validation).mean()

        # Almacenar el resultado
        results[(min_samples, min_gain)] = accuracy

results

{(5, 0.01): 0.8192090395480226,
 (5, 0.05): 0.8192090395480226,
 (5, 0.1): 0.8192090395480226,
 (5, 0.2): 0.8350282485875706,
 (5, 0.3): 0.8259887005649718,
 (10, 0.01): 0.831638418079096,
 (10, 0.05): 0.831638418079096,
 (10, 0.1): 0.831638418079096,
 (10, 0.2): 0.8293785310734463,
 (10, 0.3): 0.8271186440677966,
 (20, 0.01): 0.831638418079096,
 (20, 0.05): 0.831638418079096,
 (20, 0.1): 0.831638418079096,
 (20, 0.2): 0.8338983050847457,
 (20, 0.3): 0.8271186440677966,
 (40, 0.01): 0.8338983050847457,
 (40, 0.05): 0.8338983050847457,
 (40, 0.1): 0.8338983050847457,
 (40, 0.2): 0.8338983050847457,
 (40, 0.3): 0.8271186440677966,
 (80, 0.01): 0.8192090395480226,
 (80, 0.05): 0.8192090395480226,
 (80, 0.1): 0.8192090395480226,
 (80, 0.2): 0.8192090395480226,
 (80, 0.3): 0.8124293785310734}

### Nuevo entrenamiento con mejores hiperparámetros

In [ ]:
X_train_new = pd.concat([X_train, X_validation], ignore_index=True)
y_train_new = pd.concat([y_train, y_validation], ignore_index=True)

decision_tree = ID3TreeClassifier(min_samples_split=5, min_split_gain=0.2)
decision_tree.fit(X_train_new, y_train_new)

test_data_predictions = decision_tree.predict(X_test)

accuracy = (test_data_predictions == y_test).mean()
accuracy

0.8406779661016949

### Comparación con scikit-learn

In [ ]:
# Codificar etiquetas de categorías para y_train y y_test
label_encoder = LabelEncoder()
y_train_new_encoded = label_encoder.fit_transform(y_train_new)
y_test_encoded = label_encoder.transform(y_test)

# Entrenar y evaluar DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=10)
dt_classifier.fit(X_train_new, y_train_new_encoded)
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test_encoded, dt_predictions)

# Entrenar y evaluar RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=10)
rf_classifier.fit(X_train_new, y_train_new_encoded)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test_encoded, rf_predictions)

dt_accuracy, rf_accuracy

(0.8338983050847457, 0.8926553672316384)

## 3. Experimentación

En la Tabla 1, se presentan los distintos resultados de accuracy obtenidos con el modelo entrenado en el conjunto de entrenamiento y evaluado en el conjunto de validación, considerando distintas combinaciones de hiperparámetros.

<table>
  <tr>
    <th>min_samples_split \ min_split_gain</th>
    <th>0.01</th>
    <th>0.05</th>
    <th>0.10</th>
    <th>0.20</th>
    <th>0.30</th>
  </tr>
  <tr>
    <th>5</th>
    <td>0.819</td>
    <td>0.819</td>
    <td>0.819</td>
    <td><b>0.835</b></td>
    <td>0.826</td>
  </tr>    
  <tr>
    <th>10</th>
    <td>0.832</td>
    <td>0.832</td>
    <td>0.832</td>
    <td>0.829</td>
    <td>0.827</td>
  </tr>
  <tr>
    <th>20</th>
    <td>0.832</td>
    <td>0.832</td>
    <td>0.832</td>
    <td>0.834</td>
    <td>0.827</td>
  </tr>
  <tr>
    <th>40</th>
    <td>0.834</td>
    <td>0.834</td>
    <td>0.834</td>
    <td>0.834</td>
    <td>0.827</td>
  </tr>
  <tr>
    <th>80</th>
    <td>0.819</td>
    <td>0.819</td>
    <td>0.819</td>
    <td>0.819</td>
    <td>0.812</td>
  </tr>
  <caption>Tabla 1 - Accuracy del modelo para distintos valores de hiperparámetros</caption>
</table>


Se puede ver que el mayor valor de accuracy se obtiene para min_samples_split=5 y min_split_gain=0.2.  

A partir de esto se entrena un nuevo modelo sobre el conjunto de entrenamiento + validación, con dichos hiperparámetros.
En la Tabla 2 se compara el resultado sobre el conjunto de evaluación con los modelos de scikit-learn mencionados anteriormente.


<table>
  <tr>
    <th>ID3TreeClassifier</th>
    <th>DecisionTreeClassifier</th>
    <th>RandomForestClassifier</th>
  </tr>
  <tr>
    <td>0.841</td>
    <td>0.834</td>
    <td>0.893</td>
  </tr>    
  <caption>Tabla 2 - Accuracy de los distintos modelos entrenados</caption>
</table>

## 4. Conclusión

Los hiperparámetros utilizados en el algorítmo ID3 tienen impacto directo en la forma del árbol resultante del entrenamiento. A mayores valores, las condiciones de poda se cumplen con más facilidad, resultando en un árbol con menos niveles. Por el contrario, si se utilizara la combinación (min_samples_split = 1, min_split_gain = 0) se obtendría un árbol completamente sobreajustado a los datos de entrenamiento.

La validación con diferentes combinaciones de hiperparámetros permite encontrar el punto de equilibrio entre el subajuste y el sobreajuste. Se puede ver incluso que con esa selección, el modelo logra un valor de accuracy mayor al clasificador *DecisionTreeClassifier* de scikit-learn.

Finalmente se puede observar que el clasificador *RandomForestClassifier* es el que obtiene el valor de accuracy más alto, lo cual es un resultado esperable por la naturaleza del algoritmo Random Forest.